# Data Intuition (Cont'd)

Putting it all together:
- pandas
- data wrangling
- data intuition
- descriptive statistics
- wisdom of the crowds

We'll use your responses to the Fermi estimation Google Form to do this.

## Setup

In [ ]:
# import working with data libraries
import pandas as pd
import numpy as np

## The Data

In [ ]:
# read data into Python
df = pd.read_csv('data/fermi_sp20.csv')

In [ ]:
# take a look at the data
df

## Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### iClicker Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

## Brainstorming

What considerations do we have to make about these data?

- handling non-numeric answers (i.e. "lots")
- handle units in answer
- unreasonably large/small number (outliers)? (not thoughtful, or joking)
- scientific notation

In [ ]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

In [ ]:
# check type of each Series (column)
df.dtypes

In [ ]:
# this cell will produce an error
df['hair_growth'].median()

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

In [ ]:
# take a look at unique values
df["hair_growth"].unique()

In [ ]:
# standardize height column
def standardize_hair(string):
    
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()

    # take care of unique cases
    string = string.replace("~3 inches, so 7.62 cm", "7.62")
    string = string.replace("2.0 x 10^-1 m", "20")
    string = string.replace("10^1.2", "15.8")
    string = string.replace("10^(1.2)", "15.8")
    string = string.replace("0.5 in/month", "6")
    
    # remove commas
    string = string.replace(",", "")
    
    # take care of inclded unit cases
    string = string.replace("cm/ye", "")
    string = string.replace("a ye", "")
    string = string.replace("cm/yr", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")
    string = string.replace("ar", "")
    string = string.replace("per ye", "")


    # handle scientific notation
    string = string.replace("^2", "0")
    string = string.replace("^1", "")
    string = string.replace("10^(-2)", "0.1")
    string = string.replace("10^-2", "0.1")

    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

In [ ]:
df['hair_growth'].describe()

In [ ]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

In [ ]:
df['crammed'].unique()

In [ ]:
# standardize crammed column
def standardize_crammed(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of inclded unit cases
    string = string.replace("(km^2)", "")
    string = string.replace("km^2", "")
    string = string.replace("km2", "")
    string = string.replace("km²", "")
    string = string.replace("km", "")
    string = string.replace("squared", "")
    string = string.replace("sq", "")
    
    # take care of various scientific notation cases
    string = string.replace("3.2*10^11", "320000000000")
    string = string.replace("3.0 x 10 ^9", "3000000000")
    string = string.replace("(10^8 - 10^3) =", "")
    string = string.replace("3*10^6", "3000000")
    string = string.replace("7m", "7000000")
    string = string.replace("7*(10^3)", "7000")
    string = string.replace("10^(3.5)", "3162")
    string = string.replace("4 × 10^6", "3162")


    # take care of general scientific notation
    string = string.replace("8x10^8", "800000000")
    string = string.replace("x10^2", "00")
    string = string.replace("*(10^3)", "000")

    string = string.replace("10^3", "1000")
    string = string.replace("10^4", "10000")
    string = string.replace("10^5", "100000")
    string = string.replace(" × 10^6", "000000")
    string = string.replace("10^6", "1000000")
    string = string.replace("*10^7", "0000000")
    string = string.replace("10^7", "10000000")
    string = string.replace("*10^8", "100000000")
    string = string.replace("10^8", "100000000")
    string = string.replace(" x 10^9", "000000000")
    string = string.replace("x 10^9", "000000000")

    string = string.replace("10^9", "1000000000")
    string = string.replace("10^10", "1000000000")
    string = string.replace("10^11", "00000000000")
    string = string.replace("10^15", "1000000000000000")
    string = string.replace(" x 10^9", "000000000")
    string = string.replace("10x1000000", "10000000")

    
    # take care of unique word cases
    string = string.replace(" million", "000000")
    string = string.replace("australia", "7692024")
    string = string.replace(" billion", "000000000")
    string = string.replace("3 earths", "510072000")

    # take care of case with space in middle
    string = string.replace("100 000", "100000")


    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

In [ ]:
df.dtypes

In [ ]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
df['SAN_NYC'].unique()

In [ ]:
# standardize distance column
def standardize_distance(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of special/unclear cases
    string = string.replace("10^2 (not actually 100 but close to half)", "NaN")
    string = string.replace("lots", "NaN")
    
    # remove units
    string = string.replace("days", "")
    

    # take care of non-days answers
    string = string.replace("1month", "30")
    string = string.replace("1 month", "30")
    string = string.replace("one month", "30")
    string = string.replace("3 months", "90")
    string = string.replace("3 month", "90")
    string = string.replace("4 months", "120")
    string = string.replace("6 months", "180")
    string = string.replace("1 year", "365")
    string = string.replace("500 hrs", "21")
    string = string.replace("650 hours", "27")
    string = string.replace("750 hours", "31")
    string = string.replace("900 hours", "38")
    
    # take care of 'punctuation'
    string = string.replace("~", "")
    string = string.replace(",", "")
    
    # take care of scientific notation 
    string = string.replace("10^2", "100")
    string = string.replace("10^3", "1000")
    string = string.replace("10^4", "10000")
    string = string.replace("10^9", "1000000000")
    string = string.replace("10^(2.5)", "316")

   
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["SAN_NYC"] = df["SAN_NYC"].apply(standardize_distance)
df["SAN_NYC"].unique()

In [ ]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

## Take Home Messages

1. Look at your data! (People behave unexpectedly.)
2. Get your data into a tidy data format.
3. Fix (Standardize) your data - or, better yet, collect better data!
4. Use your intuition/knowledge/information out there to consider if values make sense.